> 
>># **Cryptographie appliqué : *Projet 3***
>>>>>>>>>>>>>> &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;`Rida VERDU | Groupe 46`
>>## **Les clées RSA publique**
>>> Une **clé publique** a deux composants principaux :<br>
>>> &emsp;&emsp;- Modulus <br>
>>> &emsp;&emsp;- Exposant<br>
>>> <br>
>>> Le **Modulus** est la somme de p et q (`N = p*q`) deux nombres premiers.<br>
>>> La taille de bit défini pendant la création de la clée correspond a celle de N.<br>
>>> Puis il y a les exposants e et d *(respectivement encrypt, decrypt)* où e serant dans la grande majoritée des cas 65537<br>
>>> car il s'agit du meilleur rapport sécurité/rapidité. <br>
>>> Puis les clé sont la jointure entre le **modulus** et les valeurs **e & d** <br> 
>>> &emsp;&emsp;Clée Publique : (e, N)<br>
>>> &emsp;&emsp;Clée Privée : (d, N)<br>
>>> #### **Etape de création des clée publique :**
>>> - Génération de p et q premiers... [Primality test](https://en.wikipedia.org/wiki/Primality_test)
>>> - Calcul de N : `N = p*q`
>>> - Calcul du *Euler Totient* : `T = (p-1) * (q-1)` <br>
>>> &emsp;&emsp;&emsp;&emsp; Autre appelations : `T = φ(n) = Φ(n)` <br>
>>> - Choix de e :  On choisi 65537 par defaut <br>
>>> &emsp;&emsp;&emsp;&emsp; Il faut que `1 < e < T` et que T et e sois copremier *(`gcd(e, T) == 1`)* <br>
>> ### Implémentation en **python**
>>> #### *Librairies* 
>>> &emsp;&emsp;&emsp;&emsp; Librairies utilisé tout au long du fichier <br>
>>> &emsp;&emsp;&emsp;&emsp;


In [1]:
import rsa
import sqlite3
from math import gcd

>>> #### *Création de clées* 
>>> &emsp;&emsp;&emsp;&emsp; Nous n'utiliserons que les clé publiques, <br>
>>> &emsp;&emsp;&emsp;&emsp; avec la fonction `newkeys` de la librairie `rsa` <br>
>>> &emsp;&emsp;&emsp;&emsp; on prend le **modulus** et l'**exposant** de le clée **publique**. <br>
>>> &emsp;&emsp;&emsp;&emsp; Pour finir on retourne un tuple du format souhaité <br>
>>> &emsp;&emsp;&emsp;&emsp;

In [2]:
def genKey(bitsize:int) -> tuple:
        (pubkey, privkey) = rsa.newkeys(bitsize)
        modulus = pubkey.n
        exponant = pubkey.e
        return (modulus, exponant, bitsize)
    
print(genKey(128))

(224578856187697349428523716769916890459, 65537, 128)


>>> #### *Génération de masse* 
>>> &emsp;&emsp;&emsp;&emsp; Mes fonctions font partie d'une class **Generator** qui traite les clées <br>
>>> &emsp;&emsp;&emsp;&emsp; à l'aide une base de donnée sqlite (déclaré plus tard dans le jupyter). <br>
>>> &emsp;&emsp;&emsp;&emsp; Cette fonction va generer a l'aide de la fonction précédente le nombre de clées souhaité des tailles listés en paramètre<br>
>>> &emsp;&emsp;&emsp;&emsp;

In [3]:
def massKeyGen(numOfKey:int, bitsize:list = [128, 256, 512, 1024, 2048]) -> list:
        keyList = []
        for size in bitsize:
            print(f"Printing {size}bit keys...", end="\n")
            for keyIndex in range(numOfKey):
                print(f" Generating... {int((keyIndex / numOfKey)*100)}%", end="\r")
                keyList.append(genKey(size))
        return keyList

for key in massKeyGen(1000, [128, 256]):
    print(key)

Printing 128bit keys...
Printing 256bit keys...
(197532847977526534362903618133748502977, 65537, 128)
(173353241337250224528758574139035818813, 65537, 128)
(172727048907192278655780315576364217851, 65537, 128)
(174600620711371719659158873924464956113, 65537, 128)
(292934047151466648228832290969342619753, 65537, 128)
(202723451504102537267258546374598875333, 65537, 128)
(297605034467473294441737336723733226431, 65537, 128)
(262850210715050048570391460818163048927, 65537, 128)
(235119088560983329453654863419542014437, 65537, 128)
(188479178464602571378614516721743962671, 65537, 128)
(207752815208313003737002298607989947807, 65537, 128)
(235311569382772959428329539881017185139, 65537, 128)
(175994910834327119473895704589673051083, 65537, 128)
(238065809682400867774417822443526053403, 65537, 128)
(220719856799172146175608441890042920679, 65537, 128)
(204077983354949023871311328498496242953, 65537, 128)
(188144359347830547405973251725916600977, 65537, 128)
(251148967810017013617030891169933

>>> #### *Arrangement du code en une class **Generator*** 
>>> &emsp;&emsp;&emsp;&emsp; Pour avoir un script plus ordonné j'implemente les fonctions dans une class **Generator**<br>
>>> &emsp;&emsp;&emsp;&emsp; afin de mieu faire j'implemente une gestion avec base de donnée Sqlite. <br>
>>> &emsp;&emsp;&emsp;&emsp; La methode `_initBDD` va crée dans la base de donnée `KeyStore.sqlite` une table Keys,<br>  
>>> &emsp;&emsp;&emsp;&emsp; avec la methode `InsertKey` je peux inserer dans la base de donnée initialisé les valeurs des tuples,<br>
>>> &emsp;&emsp;&emsp;&emsp; généré avec la methode `genKey`. <br>
>>> &emsp;&emsp;&emsp;&emsp; Pour finir j'ai une methode `close` pour cloturer les connections <br>
>>> &emsp;&emsp;&emsp;&emsp;

In [5]:
class Generator:
    def __init__(self):
        self.bddName = "KeyStore_SAMPLE.sqlite"
        self.bdd = self._initBDD()
        self.bddCursor = self.bdd.cursor()
        
    def _initBDD(self, database:str = None) -> sqlite3.Connection:
        database = self.bddName if database is None else database
        bdd =  sqlite3.connect(database)
        cursor = bdd.cursor()
        cursor.execute("""CREATE TABLE IF NOT EXISTS Keys (
            id INTEGER UNIQUE PRIMARY KEY AUTOINCREMENT,
            modulus varchar(3000) NOT NULL, 
            exponant varchar(10) NOT NULL,
            bitsize INT NOT NULL)
        """)
        bdd.commit()
        return bdd
    
    def InsertKey(self, key:tuple):
        """
        Insert a tuple of a specific form into the 'self.bddName'
        
            (modulus, exponant, bitsize) -> tuple
        """
        (modulus, exponant, bitsize) = key
        self.bddCursor.execute(f"""INSERT INTO Keys (modulus,exponant,bitsize) VALUES (
                            "{str(modulus)}", 
                            "{str(exponant)}", 
                            "{str(bitsize)}")
        """)
        self.bdd.commit()
    
    def close(self):
        self.bddCursor.close()
        self.bdd.close()
        
    @staticmethod
    def genKey(bitsize:int) -> tuple:
        (pubkey, privkey) = rsa.newkeys(bitsize)
        modulus = pubkey.n
        exponant = pubkey.e
        return  (modulus, exponant, bitsize)
    
    @staticmethod
    def massKeyGen(KeyGen:'Generator', numOfKey:int, bitsize:list = [128, 256, 512, 1024, 2048]):
        for size in bitsize:
            print(f"Printing {size}bit keys...", end="\n")
            for keyIndex in range(numOfKey):
                print(f" Generating... {int((keyIndex / numOfKey)*100)}%", end="\r")
                KeyGen.InsertKey(KeyGen.genKey(size))
                
if __name__ == "__main__":
    KeyGen = Generator()
    Generator.massKeyGen(KeyGen, 1000, [128, 256])
    KeyGen.close()

>>> ## ***Detection de dupliqués*** 
>>> &emsp;&emsp;&emsp;&emsp; La detection de dupliqués est facilité avec l'utilisation de `SQL` <br>
>>> &emsp;&emsp;&emsp;&emsp; Simple en python, mais encore plus en `SQL`<br>
>>> &emsp;&emsp;&emsp;&emsp; Je vais utilisé la fonction *COUNT()* pour counter le nombre d'occurence, <br>
>>> &emsp;&emsp;&emsp;&emsp; puis avec la contrainte <br> 
>>> &emsp;&emsp;&emsp;&emsp;

In [6]:
def CheckDuplicate(generator:'Generator') -> list:
    """
    Return list of Tuple
    
    [  (<modulus>, <iteration>, <bitsize>), ...  ]
    """
    duplicates = []
    duplicateFinderPayload = """
    SELECT modulus, COUNT(modulus), bitsize
        FROM Keys
    GROUP BY modulus, bitsize
        HAVING COUNT(modulus) > 1"""
    
    for row in generator.bddCursor.execute(duplicateFinderPayload).fetchall():    
        duplicates.append((row[0], row[1], row[2]))
    
    if len(duplicates):
        print("Duplicate has been found !")
        for duplicate in duplicates:
            print(f"\tModulus : {duplicate[0]} | Iteration : {duplicate[1]} | BitSize : {duplicate[2]}")
    
    generator.bdd.commit()
    return duplicates

KeyGen = Generator()
print(CheckDuplicate(KeyGen))

[]

>>> ## ***Application du Batch GCD*** 
>>> &emsp;&emsp;&emsp;&emsp; Le **`Batch GCD`** est une des faiblesse du chiffrement RSA qui repose sur la faible entropie <br>
>>> &emsp;&emsp;&emsp;&emsp; si on obtient un diviseur commun alors dans ce cas on a `p` ou `q` <br>
>>> &emsp;&emsp;&emsp;&emsp; ce qui rend l'attaque par bruteforce plus rapide et donc le crackage de la clée. <br>
>>> &emsp;&emsp;&emsp;&emsp; Dans le code ci-dessous, je charge les modulus des clées stocké dans le base de donnée <br>
>>> &emsp;&emsp;&emsp;&emsp; une fois cette tâche effectuée je compare chaque modulus avec la fonction `gcd()` de la librairie math <br>
>>> &emsp;&emsp;&emsp;&emsp; si le diviseur commun est plus grand que 1 dans ce cas ce dernier est donc considéré comme faillible <br>
>>> &emsp;&emsp;&emsp;&emsp;

In [9]:
def batch_gcd(generator:'Generator'):
    modulusList = []
    modulusListPayload = "SELECT modulus FROM Keys"
    
    print("Chargement des modulus...")
    for row in generator.bddCursor.execute(modulusListPayload).fetchall():
        print(f"modulus : {row[0][:40]+'...'}", end="\r")
        modulusList.append(row[0])
        
    print(f"\nTerminé ! {len(modulusList)} modulus chargés")
    print("Commencement de l'attaque gcd !")
    diviseursCommuns = []
    indexPrincipal = 0
    index = 0
    
    for modulusDeReference in modulusList:
        modulusDeReference = int(modulusDeReference)
        for modulusSecond in modulusList[indexPrincipal+1:]:
            modulusSecond = int(modulusSecond)
            print(f"Nombre de diviseur commun trouvé : {len(diviseursCommuns)} | Passage : {index}", end="\r")
            index += 1
            if gcd(modulusDeReference, modulusSecond) > 1:
                diviseursCommuns.append((modulusDeReference, modulusSecond))
        indexPrincipal +=1
        
    if len(diviseursCommuns):
        exportFilename = "DiviseursCommuns.txt"
        with open(exportFilename, "w") as fichierDiviseursCommuns:
            for coupleDeDiviseurCommun in diviseursCommuns:
                fichierDiviseursCommuns.write(f"{coupleDeDiviseurCommun[0]} {coupleDeDiviseurCommun[1]}")        
        print(f"Liste des couples ayant un diviseurs commun exporté dans {exportFilename}")
    else: 
        print("Aucun diviseurs commun n'a été trouvé \n")
        
KeyGen = Generator()
batch_gcd(KeyGen)
print("Diviseurs communs écrit dans DiviseursCommuns.txt s'il y en a.")
KeyGen.close()

Chargement des modulus...
modulus : 7350694644728345224671949369824169716604...
Terminé ! 1000 modulus chargés
Commencement de l'attaque gcd !
Aucun diviseurs commun n'a été trouvé| Passage : 499499
Diviseurs communs écrit dans DiviseursCommuns.txt


>> # **Conclusion**
>> &emsp;&emsp;&emsp; J'ai beaucoup apprecié faire ce projet, ça m'a fait decouvrir plus en profondeur le monde de la cryptographie. <br>
>> &emsp;&emsp;&emsp; J'ai toujours eu beaucoup d'apprehension envers ce monde car peur du côté mathematique et du manque de bonne expliquation, <br>
>> &emsp;&emsp;&emsp; mais le fait de me plonger dedans m'a vraiment fait aimer ce domaine. <br>
>> &emsp;&emsp;&emsp; J'en retire que du positif même si je n'ai pas eu toutes les expliquations que je cherchais manque de ressources, <br>
>> &emsp;&emsp;&emsp; je suis motivié a continuer a decouvrires d'autres methodes (je viens de refaire l'implementation ECCDH pour un projet personnel) <br>
>> &emsp;&emsp;&emsp;
>>>>>>>>>>>>>> &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;`Rida VERDU | Groupe 46`